In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from collections import defaultdict
from sklearn.svm import SVC  
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import sklearn
import pickle
import importlib.util

In [2]:
# importing template file 
spec = importlib.util.spec_from_file_location("Template", "/home/admin1/PycharmProjects/ML/Week10/Template/template.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
# creating object of Template class
temp = foo.Template()

In [3]:
# read file
df=pd.read_csv('Data/1625Data.txt',names=['octamers','flags'])
df.head()

,octamers,flags
0,SLNLRETN,1
1,AECFRIFD,1
2,HLVEALYL,1
3,TQIMFETF,1
4,AEELAEIF,1


In [4]:
df.shape

(1625, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625 entries, 0 to 1624
Data columns (total 2 columns):
octamers    1625 non-null object
flags       1625 non-null int64
dtypes: int64(1), object(1)
memory usage: 25.5+ KB


In [6]:
df.describe()

,flags
count,1625.000000
mean,-0.538462
std,0.842909
min,-1.000000
25%,-1.000000
50%,-1.000000
75%,-1.000000
max,1.000000


In [7]:
# checks for null values
df.isnull().sum()

octamers    0
flags       0
dtype: int64

In [8]:
# checks for duplicate values
df.duplicated().sum()

0

In [9]:
df.replace({-1:0,1:1},inplace=True)
df.head()

,octamers,flags
0,SLNLRETN,1
1,AECFRIFD,1
2,HLVEALYL,1
3,TQIMFETF,1
4,AEELAEIF,1


In [10]:
df.head()

,octamers,flags
0,SLNLRETN,1
1,AECFRIFD,1
2,HLVEALYL,1
3,TQIMFETF,1
4,AEELAEIF,1


In [11]:
# Seperate all amino acids
octamers = np.array([[df["octamers"][i][j] for i in range(df.shape[0])] for j in range(8)])
print(octamers)


[['S' 'A' 'H' ... 'Q' 'D' 'S']
 ['L' 'E' 'L' ... 'A' 'S' 'K']
 ['N' 'C' 'V' ... 'I' 'A' 'D']
 ...
 ['E' 'I' 'L' ... 'A' 'E' 'A']
 ['T' 'F' 'Y' ... 'L' 'E' 'E']
 ['N' 'D' 'L' ... 'Q' 'D' 'I']]


In [12]:
# Store the seperated amino acids into a dataframe
df1=pd.DataFrame(octamers.T, columns=list('ABCDEFGH'))
print(df1)

      A  B  C  D  E  F  G  H
0     S  L  N  L  R  E  T  N
1     A  E  C  F  R  I  F  D
2     H  L  V  E  A  L  Y  L
3     T  Q  I  M  F  E  T  F
4     A  E  E  L  A  E  I  F
5     P  F  I  F  E  E  E  P
6     P  I  V  G  A  E  T  F
7     E  T  T  A  L  V  C  D
8     G  G  V  Y  A  T  R  S
9     D  A  I  N  T  E  F  K
10    D  D  L  F  F  E  A  D
11    S  F  I  G  M  E  S  A
12    P  T  L  L  T  E  A  P
13    D  Q  I  L  I  E  I  C
14    A  Q  T  F  Y  V  N  L
15    Q  I  T  L  W  Q  R  P
16    G  S  H  L  V  E  A  L
17    R  E  A  F  R  V  F  D
18    Y  E  E  F  V  Q  M  M
19    G  Q  V  N  Y  E  E  F
20    L  P  V  N  G  E  F  S
21    E  L  E  F  P  E  G  G
22    V  E  V  A  E  E  E  E
23    D  T  V  L  E  E  M  S
24    G  D  A  L  L  E  R  N
25    A  A  K  F  E  R  Q  H
26    M  D  S  S  T  S  A  A
27    S  S  N  Y  C  N  Q  M
28    T  P  G  S  R  N  L  C
29    G  S  S  K  Y  P  N  C
...  .. .. .. .. .. .. .. ..
1595  P  S  I  N  N  E  T  P
1596  P  D  I  V  I  Y  Q  Y
1597  W  Y  Q 

In [13]:
df=pd.concat([df1,df],axis=1)
df.drop(columns=['octamers'],inplace=True)
df.head()

,A,B,C,D,E,F,G,H,flags
0,S,L,N,L,R,E,T,N,1
1,A,E,C,F,R,I,F,D,1
2,H,L,V,E,A,L,Y,L,1
3,T,Q,I,M,F,E,T,F,1
4,A,E,E,L,A,E,I,F,1


In [14]:
train,test=temp.split(df,0.2)

In [15]:
print("train",train.shape)
print("test",test.shape)

train (1300, 9)
test (325, 9)


In [16]:
# saving datasets into csv filesS
temp.save_csv(test,'test_data.csv')
temp.save_csv(train,'train_data.csv')


In [17]:
# loading training data csv file
train_df = temp.read_file('train_data.csv')
train_df.head()

,A,B,C,D,E,F,G,H,flags
0,K,Q,G,Q,G,Q,W,T,0
1,S,R,S,L,Y,A,S,S,1
2,L,W,Y,Q,L,E,K,E,0
3,A,S,V,N,C,A,K,K,0
4,E,M,E,E,N,F,A,V,0


In [18]:
# splitting training data into train and cross validation dataset 
train_data,cv_data=temp.split(train_df,0.2)

In [19]:
# saving cross validation data into csv file
temp.save_csv(cv_data,'cv_data.csv')

In [20]:
# separating features and labels of training dataset
x_train=train_data.iloc[:,:-1].values
y_train=train_data.iloc[:,8].values

In [21]:
x_train=pd.DataFrame(x_train)

In [22]:
x_train=temp.oneHotEncoding(x_train)

In [23]:
print(x_train.shape)

(1040, 160)


In [24]:
# feature scaling
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
# test=sc.transform(test)

In [25]:
# fit model

classifier = SVC(kernel='poly', degree=8)  
classifier.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=8, gamma='auto_deprecated',
  kernel='poly', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [26]:
y_pred = classifier.predict(x_train)
df = pd.DataFrame({'Actual': y_train, 'Predicted': y_pred})  
df.head()

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,1,0
4,0,0


In [27]:
# making confusion matrix
cm= confusion_matrix(y_train,y_pred)
print(cm)

[[811   0]
 [202  27]]


In [28]:
# loading cross validation dataset file
cv_data = temp.read_file('cv_data.csv')
cv_data.head()

,A,B,C,D,E,F,G,H,flags
0,P,R,V,A,L,A,M,T,1
1,A,R,H,L,R,E,Y,Q,0
2,T,K,K,L,V,V,Q,P,0
3,A,A,M,K,R,H,G,L,0
4,G,A,V,S,L,A,M,T,1


In [29]:
# separating features and labels of training dataset
x_cv=cv_data.iloc[:,:-1].values
y_cv=cv_data.iloc[:,8].values
x_cv.shape

(260, 8)

In [30]:
x_cv=pd.DataFrame(x_cv)
x_cv.shape

(260, 8)

In [31]:
x_cv=temp.oneHotEncoding(x_cv)

In [32]:
x_cv.shape

(260, 160)

In [33]:
# feature scaling
x_cv=sc.fit_transform(x_cv)

In [34]:
class SupportVector:
    
    def get_predictions(self,x):        
        # getting prediction values
        y_pred = classifier.predict(x)
        return y_pred
    
    def create_confusion_matrix(self,y,y_pred):
        # making confusion matrix
        cm= confusion_matrix(y,y_pred)
        return cm
    
    def get_accuracy(self,y_train,y_pred):
        Accuracy = sklearn.metrics.balanced_accuracy_score(y_train,y_pred)*100
        return Accuracy
        
def main():
    # creates class object 
    obj = SupportVector()
    y_pred_train = obj.get_predictions(x_train)
    y_pred_test = obj.get_predictions(x_cv)
    
    cm_train=obj.create_confusion_matrix(y_train,y_pred_train)
    print("train matrix\n",cm_train)
    
    cm_cv=obj.create_confusion_matrix(y_cv,y_pred_test)
    print("cv matrix\n",cm_cv)
    
    acc_train = obj.get_accuracy(y_train,y_pred_train)
    print("Accuracy of train data =",acc_train)
    
    acc_test = obj.get_accuracy(y_cv,y_pred_test)
    print("Accuracy of test data =",acc_test)
    
    if acc_train >= 50 and acc_test >=  50:
        fileObject = open("train_data.pkl",'wb')
        pickle.dump(classifier,fileObject)   
        pickle.dump(sc,fileObject)
        # here we close the fileObject
        fileObject.close()

        
if __name__ == '__main__':
    main()

train matrix
 [[811   0]
 [202  27]]
cv matrix
 [[208   0]
 [ 52   0]]
Accuracy of train data = 55.895196506550214
Accuracy of test data = 50.0
